## QUANTIFOIL CIRCLE OPTIMIZE
# Run the circle optimize code to calculate circularity metrics for tilt correction and least squares fit for a visual check of a circle fit.

1. Run circle_optimize function to prepare for mask generation, circularity calculation and least squares fit.

In [ ]:
def circle_optimize(input_folder, epsilon_ratio=0.001):
    """
    Input:
        input_folder: folder containing ONLY raw images to analyze
                      (e.g. .../Code/Data)
    Output folders (created in parent of input_folder, e.g. .../Code):
        - mask_overlay/
        - mask_polygon/
        - circle_optimize_data/
            - least_squares_images/
            - least_squares_error_plots/
        - circle_optimize_data/circle_optimize_table.csv
    """

    import os
    import cv2
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from skimage.measure import ransac, CircleModel
    from scipy.optimize import least_squares

    # -----------------------------
    # Paths (outputs next to code)
    # -----------------------------
    input_folder = os.path.abspath(input_folder)
    base_folder = os.path.dirname(input_folder)  # parent of Data (= code folder)

    mask_overlay_folder = os.path.join(base_folder, "mask_overlay")
    mask_polygon_folder = os.path.join(base_folder, "mask_polygon")
    data_folder = os.path.join(base_folder, "circle_optimize_data")
    ls_image_folder = os.path.join(data_folder, "least_squares_images")
    ls_err_plot_folder = os.path.join(data_folder, "least_squares_error_plots")

    for f in [mask_overlay_folder, mask_polygon_folder,
              data_folder, ls_image_folder, ls_err_plot_folder]:
        os.makedirs(f, exist_ok=True)

    # -----------------------------
    # Collect image filenames only
    # -----------------------------
    valid_exts = (".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp")

    image_names = [
        f for f in os.listdir(input_folder)
        if os.path.isfile(os.path.join(input_folder, f))
        and f.lower().endswith(valid_exts)
    ]

    if not image_names:
        raise ValueError(f"No image files found in: {input_folder}")

    df = pd.DataFrame(image_names, columns=["image"])

    # -----------------------------
    # Helper: make masks for each image
    # -----------------------------
    for filename in image_names:
        input_path = os.path.join(input_folder, filename)

        img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Could not read image: {input_path}")

        # 1. Blur
        blur = cv2.GaussianBlur(img, (7, 7), 0)

        # 2. Threshold (invert because object is dark)
        _, thresh = cv2.threshold(
            blur, 0, 255,
            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
        )

        # 3. Morphological closing
        kernel = np.ones((9, 9), np.uint8)
        closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        # 4. Find contours
        contours, _ = cv2.findContours(
            closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        if not contours:
            raise ValueError(f"No contours found in image: {input_path}")

        # 5. Largest contour
        cnt = max(contours, key=cv2.contourArea)

        # 6. Polygon approximation
        eps = epsilon_ratio * cv2.arcLength(cnt, True)
        poly = cv2.approxPolyDP(cnt, eps, True)

        # 7. Overlay mask on image
        overlay = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        mask_tmp = np.zeros_like(img, dtype=np.uint8)
        cv2.fillPoly(mask_tmp, [poly], 255)
        overlay[mask_tmp == 255] = (0, 0, 255)

        name, ext = os.path.splitext(filename)
        mask_overlay_path = os.path.join(mask_overlay_folder,
                                         f"mask_overlay_{name}{ext}")
        cv2.imwrite(mask_overlay_path, overlay)

        # 8. Save polygon mask (binary PNG)
        mask = np.zeros_like(img, dtype=np.uint8)
        cv2.fillPoly(mask, [poly], 255)
        mask_polygon_path = os.path.join(mask_polygon_folder,
                                         f"mask_polygon_{name}.png")
        cv2.imwrite(mask_polygon_path, mask)

    # -----------------------------
    # Helper: circularity measure
    # -----------------------------
    def compute_circularity(image_name):
        input_path = os.path.join(input_folder, image_name)

        img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return None

        blur = cv2.GaussianBlur(img, (7, 7), 0)
        _, thresh = cv2.threshold(
            blur, 0, 255,
            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
        )

        contours, _ = cv2.findContours(
            thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        if not contours:
            return None

        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        if perimeter == 0:
            return None

        circularity = 4 * np.pi * area / (perimeter ** 2)  # 1 = perfect circle [web:13]
        return circularity

    df["circularity"] = df["image"].apply(compute_circularity)

    # -----------------------------
    # Helper: least squares circle fit
    # -----------------------------
    def compute_least_squares(image_name):
        image_path = os.path.join(input_folder, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Could not read image: {image_path}")

        name, _ = os.path.splitext(image_name)
        mask_polygon_path = os.path.join(mask_polygon_folder,
                                         f"mask_polygon_{name}.png")
        mask = cv2.imread(mask_polygon_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Could not read mask: {mask_polygon_path}")

        _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(
            mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
        if not contours:
            raise ValueError(f"No mask contour found for: {mask_polygon_path}")

        cnt = max(contours, key=cv2.contourArea)

        # Subpixel refine
        cnt = cnt.astype(np.float32)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,
                    40, 0.001)
        cv2.cornerSubPix(
            img,
            cnt,
            winSize=(5, 5),
            zeroZone=(-1, -1),
            criteria=criteria
        )

        pts = cnt.squeeze()  # (N, 2)
        if pts.ndim != 2 or pts.shape[0] < 3:
            raise ValueError(f"Not enough contour points for: {image_path}")

        # Uniform sampling to at most ~300 points
        step = max(1, len(pts) // 300)
        pts_sampled = pts[::step]
        x, y = pts_sampled[:, 0], pts_sampled[:, 1]

        data = np.column_stack([x, y])

        # Robust initial circle via RANSAC [web:131][web:137]
        model_robust, inliers = ransac(
            data,
            CircleModel,
            min_samples=3,
            residual_threshold=1.0,
            max_trials=1000
        )
        if model_robust is None or inliers is None or not np.any(inliers):
            raise ValueError(f"RANSAC failed for: {image_path}")

        x_in, y_in = data[inliers].T

        # Least squares refinement [web:13][web:132]
        def residuals(c):
            a, b, r = c
            return np.sqrt((x_in - a) ** 2 + (y_in - b) ** 2) - r

        a0, b0, r0 = model_robust.params
        res = least_squares(residuals, [a0, b0, r0])
        a, b, r = res.x

        residual = residuals([a, b, r])
        rmse = np.sqrt(np.mean(residual ** 2))
        arc_fraction = len(x_in) / len(x)

        # Plot circle overlay
        theta = np.linspace(0, 2 * np.pi, 500)
        xc = a + r * np.cos(theta)
        yc = b + r * np.sin(theta)

        plt.imshow(img, cmap='gray')
        plt.scatter(x, y, s=2, c='yellow')
        plt.plot(xc, yc, 'r', lw=2)
        plt.scatter(a, b, c='cyan')
        plt.axis('off')
        plt.tight_layout()

        ls_image_path = os.path.join(ls_image_folder,
                                     f"least_squares_{name}.png")
        plt.savefig(ls_image_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        # Plot angular error
        angles = np.arctan2(y - b, x - a)
        errors = np.sqrt((x - a) ** 2 + (y - b) ** 2) - r

        plt.scatter(angles, errors, s=5)
        plt.axhline(0, color='k')
        plt.xlabel("Angle (rad)")
        plt.ylabel("Radial error (px)")
        ls_err_path = os.path.join(ls_err_plot_folder,
                                   f"ls_error_plot_{name}.png")
        plt.savefig(ls_err_path, bbox_inches='tight', pad_inches=0.1)
        plt.close()

        return rmse, arc_fraction

    df[["LS RMSE", "LS arc coverage"]] = df["image"].apply(
        lambda name: pd.Series(compute_least_squares(name))
    )

    # Save table
    table_path = os.path.join(data_folder, "circle_optimize_table.csv")
    df.to_csv(table_path, index=False)

    return df


2. Load folder directory within the quotation marks and run circle_optimize function.

In [ ]:
circle_optimize(input_folder="")